In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
max_length = 512


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
class PromptResponseDataset(Dataset):
    def __init__(self, prompts, responses, tokenizer, max_length=512):
        self.encodings = tokenizer(
            [p + " " + r for p, r in zip(prompts, responses)],
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.encodings["input_ids"][idx]
        }


In [ ]:


import jsonlines

prompts = []
responses = []

with jsonlines.open('game_dataset.jsonl') as reader:
    for obj in reader:
        prompts.append(obj['prompt'])
        responses.append(obj['response'])

dataset = PromptResponseDataset(prompts, responses, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

for batch in dataloader:
    print(batch)
    break


In [ ]:
import json

dataset_path = "/content/drive/MyDrive/Bulipe/game_dataset.jsonl"
data = []

with open(dataset_path, "r") as f:
    for line in f:
        data.append(json.loads(line))

prompts = [item["prompt"] for item in data]
responses = [item["code"] for item in data]

print("Sample prompt:", prompts[0])
print("Sample response:", responses[0])


Sample prompt: Create a Flappy Bird clone where the bird jumps when space is pressed.
Sample response: <!DOCTYPE html>
<html>
<head>
  <title>Flappy Bird Clone</title>
  <style>
    canvas { background: #70c5ce; display: block; margin: 0 auto; }
  </style>
</head>
<body>
<canvas id="gameCanvas" width="320" height="480"></canvas>
<script>
  const canvas = document.getElementById('gameCanvas');
  const ctx = canvas.getContext('2d');
  let birdY = 150, gravity = 1.5, velocity = 0;
  document.addEventListener('keydown', () => velocity = -10);

  function draw() {
    ctx.clearRect(0, 0, canvas.width, canvas.height);
    velocity += gravity;
    birdY += velocity;
    ctx.fillStyle = 'yellow';
    ctx.beginPath();
    ctx.arc(50, birdY, 15, 0, Math.PI * 2);
    ctx.fill();
    requestAnimationFrame(draw);
  }
  draw();
</script>
</body>
</html>


In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Avoid padding errors

class PromptResponseDataset(Dataset):
    def __init__(self, prompts, responses, tokenizer, max_length=512):
        self.encodings = tokenizer(
            [p + " " + r for p, r in zip(prompts, responses)],
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.encodings["input_ids"][idx]
        }

dataset = PromptResponseDataset(prompts, responses, tokenizer)


In [ ]:
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM
import torch
from torch.optim import AdamW # Import AdamW from torch.optim
from tqdm import tqdm

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(3):
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Epoch 2: 100%|██████████| 1/1 [00:19<00:00, 19.34s/it, loss=1.68]


In [ ]:
output_dir = "/content/drive/MyDrive/Bulipe/finetuned_gpt2_game"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/Bulipe/finetuned_gpt2_game/tokenizer_config.json',
 '/content/drive/MyDrive/Bulipe/finetuned_gpt2_game/special_tokens_map.json',
 '/content/drive/MyDrive/Bulipe/finetuned_gpt2_game/vocab.json',
 '/content/drive/MyDrive/Bulipe/finetuned_gpt2_game/merges.txt',
 '/content/drive/MyDrive/Bulipe/finetuned_gpt2_game/added_tokens.json',
 '/content/drive/MyDrive/Bulipe/finetuned_gpt2_game/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model_dir = "/content/drive/MyDrive/Bulipe/finetuned_gpt2_game"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)

model.eval()

def generate_response(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

prompt = "Create a Flappy Bird clone where the bird jumps when space is pressed"
print(generate_response(prompt))


Create a Flappy Bird clone where the bird jumps when space is pressed.

The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird will jump when space is pressed. The bird
